# Imports

In [1]:
!pip install scipy
!pip install matplotlib

In [2]:
from scipy.stats import norm as normal
import math
import numpy as np

In [3]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [4]:
%matplotlib notebook

In [5]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pylab import MaxNLocator

# Implementations Formulas

## Price Formulas

In [6]:
def calcDs(S, K, r, sigma, t):
    d1 = (math.log(S/K) + (r + sigma**2/2)*t)/(sigma * math.sqrt(t))
    d2 = d1 - sigma * math.sqrt(t)
    return (d1,d2)

In [7]:
def calcCall(S,K,t,r,sigma):
    (d1,d2) = calcDs(S, K, r, sigma, t)
    return round(S*normal.cdf(d1) - K *math.exp(-r*t)*normal.cdf(d2),2)

def calcPut(S,K,t,r,sigma):
    (d1,d2) = calcDs(S, K, r, sigma, t)
    return round(K*math.exp(-r*t)*normal.cdf(-d2) - S * normal.cdf(-d1),2)

In [8]:
def calcOption(optionType, S, K, t, r):
    if optionType == "Put":
        return calcPut(S, K, t, r, sigma)
    if optionType == "Call":
        return calcCall(S, K, t, r, sigma)        

## Greeks

In [9]:
def calcDeltaCall(S,K, t,r, sigma):
    d1 = calcDs(S, K, r, sigma, t)[0]
    return normal.cdf(d1)

In [10]:
def calcThetaCall(S,K, t,r, sigma):
    (d1,d2) = calcDs(S, K, r, sigma, t)
    term1 = S*phi(d1,0,1)*sigma/(2*math.sqrt(t))
    term2 = r*K*math.exp(-1*r*t)*normal.cdf(d2)
    return -1*term1 - term2

In [11]:
def calcGammaCall(S,K, t,r, sigma):
    d1 = calcDs(S, K, r, sigma, t)[0]
    return phi(d1, 0,1)/(S*sigma*math.sqrt(t))

In [12]:
def calcVegaCall(S,K, t,r, sigma):
    d1 = calcDs(S, K, r, sigma, t)[0]
    term1 = S*phi(d1,0,1)
    term2 = math.sqrt(t)
    return term1 *term2

In [13]:
def calcRhoCall(S,K, t,r, sigma):
    (d1,d2) = calcDs(S, K, r, sigma, t)
    term1 = t*K*math.exp(-1*r*t)
    term2 = normal.cdf(d2)
    return term1*term2

# Utils

In [14]:
def phi(x,mu, sigma):
    return 1/math.sqrt(2*math.pi * sigma**2 )*math.exp(-(x-mu)**2/(2*(sigma**2)))
    

In [15]:
def plotPricePutCallByStrike(S,t,r,sigma):
    fig = plt.figure()
    Strikes = np.arange(50, 150)
    vals_call = [calcCall(S,K, t , r, sigma) for K in Strikes]
    vals_put = [calcPut(S,K, t , r, sigma) for K in Strikes]
    plt.plot(Strikes,vals_call, 'r', label = "Call")
    plt.plot(Strikes, vals_put, 'b', label  = "Put")
    plt.legend()
    plt.ylabel("Option Price ($)")
    plt.xlabel("Strike Price ($)")
    plt.show()

In [16]:
interactive_plot1 = interactive(plotPricePutCallByStrike,
                               S = widgets.BoundedFloatText(
                                value=100,
                                min=50,
                                max=150,
                                step=1,
                                description='Basispreis:',
                                disabled=False
                                ),
                               t = widgets.BoundedFloatText(
                                value=1,
                                min=0.1,
                                max=2,
                                step=0.1,
                                description='Laufzeit:',
                                disabled=False
                                ),
                               r = widgets.BoundedFloatText(
                                value=0.01,
                                min=0,
                                max=1,
                                step=0.01,
                                description='Zinssatz:',
                                disabled=False
                                ),
                               sigma = widgets.BoundedFloatText(
                                value=0.05,
                                min=0.01,
                                max=1,
                                step=0.01,
                                description='Volatilität:',
                                disabled=False
                                ))

In [17]:
def plotCallPriceSKfixed(t1,r1,sigma1, t2, r2, sigma2):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x = y = np.arange(90, 110, 1)
    X, Y = np.meshgrid(x, y)
    
    vectorizedPartialSK1 = np.vectorize(lambda S,K: calcCall(S,K,t1,r1,sigma1))
    vectorizedPartialSK2 = np.vectorize(lambda S,K: calcCall(S,K,t2,r2,sigma2))
    
    zs1 = np.array(vectorizedPartialSK1(np.ravel(X), np.ravel(Y)))
    Z1 = zs1.reshape(X.shape)
    
    zs2 = np.array(vectorizedPartialSK2(np.ravel(X), np.ravel(Y)))
    Z2 = zs2.reshape(X.shape)
    
    
    
    
    ax.scatter(X, Y, Z1)
    ax.scatter(X, Y, Z2)

    xa = ax.get_xaxis()
    xa.set_major_locator(MaxNLocator(integer=True))
    ya = ax.get_yaxis()
    ya.set_major_locator(MaxNLocator(integer=True))


    ax.set_xlabel('Asset Price')
    ax.set_ylabel('Strike Price')
    ax.set_zlabel('BS-Call-Price')

In [18]:
interactive_plot2 = interactive(plotCallPriceSKfixed,
                               t1 = widgets.BoundedFloatText(
    value=1,
    min=0.1,
    max=2,
    step=0.1,
    description='Laufzeit1:',
    disabled=False
),
                               r1 = widgets.BoundedFloatText(
    value=0.01,
    min=0,
    max=1,
    step=0.01,
    description='Zinssatz1:',
    disabled=False
),
                               sigma1 = widgets.BoundedFloatText(
    value=0.05,
    min=0.01,
    max=1,
    step=0.01,
    description='Volatilität1:',
    disabled=False
),
                               t2 = widgets.BoundedFloatText(
    value=1.1,
    min=0.1,
    max=2,
    step=0.1,
    description='Laufzeit2:',
    disabled=False
),
                               r2 = widgets.BoundedFloatText(
    value=0.01,
    min=0,
    max=1,
    step=0.01,
    description='Zinssatz2:',
    disabled=False
),
                               sigma2 = widgets.BoundedFloatText(
    value=0.2,
    min=0.01,
    max=1,
    step=0.01,
    description='Volatilität2:',
    disabled=False
))

In [19]:
def plotCallPriceSigmaTfixed(S1,K1,r1, S2, K2, r2):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x = y = np.arange(0.01, 1, 0.05)
    X, Y = np.meshgrid(x, y)
    
    vectorizedPartialSigmaT1 = np.vectorize(lambda sigma, t: calcCall(S1,K1,t,r1,sigma))
    zs1 = np.array(vectorizedPartialSigmaT1(np.ravel(X), np.ravel(Y)))
    Z1 = zs1.reshape(X.shape)
    
    vectorizedPartialSigmaT2 = np.vectorize(lambda sigma, t: calcCall(S2,K2,t,r2,sigma))
    zs2 = np.array(vectorizedPartialSigmaT2(np.ravel(X), np.ravel(Y)))
    Z2 = zs2.reshape(X.shape)

    ax.scatter(X, Y, Z1)
    ax.scatter(X,Y, Z2)

    xa = ax.get_xaxis()
    xa.set_major_locator(MaxNLocator(integer=True))
    ya = ax.get_yaxis()
    ya.set_major_locator(MaxNLocator(integer=True))


    ax.set_xlabel('Vola')
    ax.set_ylabel('timeToMaturity')
    ax.set_zlabel('BS-Call-Price')

In [20]:
interactive_plot3 = interactive(plotCallPriceSigmaTfixed,
                               S1 = widgets.BoundedFloatText(
    value=100,
    min=50,
    max=150,
    step=1,
    description='Basispreis1:',
    disabled=False
),
                               K1 = widgets.BoundedFloatText(
    value=110,
    min=50,
    max=150,
    step=1,
    description='Ausübungspreis1:',
    disabled=False
),
                               r1 = widgets.BoundedFloatText(
    value=0.01,
    min=0,
    max=1,
    step=0.01,
    description='Zinssatz1:',
    disabled=False
),
                               S2 = widgets.BoundedFloatText(
    value=110,
    min=50,
    max=150,
    step=1,
    description='Basispreis2:',
    disabled=False
),
                               K2 =  widgets.BoundedFloatText(
    value=115,
    min=50,
    max=150,
    step=1,
    description='Ausübungspreis2:',
    disabled=False
),
                               r2 = widgets.BoundedFloatText(
    value=0.01,
    min=0,
    max=1,
    step=0.01,
    description='Zinssatz2:',
    disabled=False
))

In [21]:
def plotDeltaStockPriceExpiry(K,r,sigma):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x = np.arange(90, 120, 0.5)
    y = np.arange(0.01, 2, 0.05)
    X, Y = np.meshgrid(x, y)
    
    vectorizedPartialDelta1 = np.vectorize(lambda S, t: calcDeltaCall(S,K, t,r, sigma))
    zs1 = np.array(vectorizedPartialDelta1(np.ravel(X), np.ravel(Y)))
    Z1 = zs1.reshape(X.shape)
    
    ax.scatter(X, Y, Z1)

    xa = ax.get_xaxis()
    xa.set_major_locator(MaxNLocator(integer=True))
    ya = ax.get_yaxis()
    ya.set_major_locator(MaxNLocator(integer=True))


    ax.set_xlabel('Asset Price')
    ax.set_ylabel('timeToMaturity')
    ax.set_zlabel('Call Delta')
    

In [22]:
interactive_plot4 = interactive(plotDeltaStockPriceExpiry,
                               K =  widgets.BoundedFloatText(
    value=110,
    min=50,
    max=150,
    step=1,
    description='Ausübungspreis:',
    disabled=False
),
                               r = widgets.BoundedFloatText(
    value=0.01,
    min=0,
    max=1,
    step=0.01,
    description='Zinssatz:',
    disabled=False
),
                               sigma = widgets.BoundedFloatText(
    value=0.05,
    min=0.01,
    max=1,
    step=0.01,
    description='Volatilität:',
    disabled=False
))

In [23]:
def plotGammaStockPriceExpiry(K,r,sigma):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x = np.arange(105, 115, 0.5)
    y = np.arange(0.01, 1, 0.02)
    X, Y = np.meshgrid(x, y)
    
    vectorizedPartialGamma1 = np.vectorize(lambda S, t: calcGammaCall(S,K, t,r, sigma))
    zs1 = np.array(vectorizedPartialGamma1(np.ravel(X), np.ravel(Y)))
    Z1 = zs1.reshape(X.shape)
    
    ax.scatter(X, Y, Z1)

    xa = ax.get_xaxis()
    xa.set_major_locator(MaxNLocator(integer=True))
    ya = ax.get_yaxis()
    ya.set_major_locator(MaxNLocator(integer=True))


    ax.set_xlabel('Asset Price')
    ax.set_ylabel('timeToMaturity')
    ax.set_zlabel('Call Gamma')

In [24]:
interactive_plot5 = interactive(plotGammaStockPriceExpiry,
                               K =  widgets.BoundedFloatText(
    value=110,
    min=50,
    max=150,
    step=1,
    description='Ausübungspreis:',
    disabled=False
),
                               r = widgets.BoundedFloatText(
    value=0.01,
    min=0,
    max=1,
    step=0.01,
    description='Zinssatz:',
    disabled=False
),
                               sigma = widgets.BoundedFloatText(
    value=0.05,
    min=0.01,
    max=1,
    step=0.01,
    description='Volatilität:',
    disabled=False
))
                               


In [25]:
def plotVegaStockPriceExpiry(K,r,sigma):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x = np.arange(80, 120, 1)
    y = np.arange(0.01, 2, 0.05)
    X, Y = np.meshgrid(x, y)
    
    vectorizedPartialVega1 = np.vectorize(lambda S, t: calcVegaCall(S,K, t,r, sigma))
    zs1 = np.array(vectorizedPartialVega1(np.ravel(X), np.ravel(Y)))
    Z1 = zs1.reshape(X.shape)
    
    ax.scatter(X, Y, Z1)

    xa = ax.get_xaxis()
    xa.set_major_locator(MaxNLocator(integer=True))
    ya = ax.get_yaxis()
    ya.set_major_locator(MaxNLocator(integer=True))


    ax.set_xlabel('Asset Price')
    ax.set_ylabel('timeToMaturity')
    ax.set_zlabel('Call Vega')

In [26]:
interactive_plot6 = interactive(plotVegaStockPriceExpiry,
                               K =  widgets.BoundedFloatText(
    value=110,
    min=50,
    max=150,
    step=1,
    description='Ausübungspreis:',
    disabled=False
),
                               r = widgets.BoundedFloatText(
    value=0.01,
    min=0,
    max=1,
    step=0.01,
    description='Zinssatz:',
    disabled=False
),
                               sigma = widgets.BoundedFloatText(
    value=0.15,
    min=0.01,
    max=1,
    step=0.01,
    description='Volatilität:',
    disabled=False
))
                               


In [27]:
def plotThetaStockPriceExpiry(K,r,sigma):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x = np.arange(105, 115, 0.1)
    y = np.arange(0.01, 0.3, 0.01)
    X, Y = np.meshgrid(x, y)
    
    vectorizedPartialTheta1 = np.vectorize(lambda S, t: calcThetaCall(S,K, t,r, sigma))
    zs1 = np.array(vectorizedPartialTheta1(np.ravel(X), np.ravel(Y)))
    Z1 = zs1.reshape(X.shape)
    
    ax.scatter(X, Y, Z1)

    xa = ax.get_xaxis()
    xa.set_major_locator(MaxNLocator(integer=True))
    ya = ax.get_yaxis()
    ya.set_major_locator(MaxNLocator(integer=True))


    ax.set_xlabel('Asset Price')
    ax.set_ylabel('timeToMaturity')
    ax.set_zlabel('Call Theta')

In [28]:
interactive_plot7 = interactive(plotThetaStockPriceExpiry,
                               K =  widgets.BoundedFloatText(
    value=110,
    min=50,
    max=150,
    step=1,
    description='Ausübungspreis:',
    disabled=False
),
                               r = widgets.BoundedFloatText(
    value=0.01,
    min=0,
    max=1,
    step=0.01,
    description='Zinssatz:',
    disabled=False
),
                               sigma = widgets.BoundedFloatText(
    value=0.05,
    min=0.01,
    max=1,
    step=0.01,
    description='Volatilität:',
    disabled=False
))                             

In [29]:
def plotRhoStockPriceExpiry(K,r,sigma):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x = np.arange(90, 150, 2)
    y = np.arange(0.01, 2, 0.05)
    X, Y = np.meshgrid(x, y)
    
    vectorizedPartialRho1 = np.vectorize(lambda S, t: calcRhoCall(S,K, t,r, sigma))
    zs1 = np.array(vectorizedPartialRho1(np.ravel(X), np.ravel(Y)))
    Z1 = zs1.reshape(X.shape)
    
    ax.scatter(X, Y, Z1)

    xa = ax.get_xaxis()
    xa.set_major_locator(MaxNLocator(integer=True))
    ya = ax.get_yaxis()
    ya.set_major_locator(MaxNLocator(integer=True))


    ax.set_xlabel('Asset Price')
    ax.set_ylabel('timeToMaturity')
    ax.set_zlabel('Call Rho')

In [30]:
interactive_plot8 = interactive(plotRhoStockPriceExpiry,
                               K =  widgets.BoundedFloatText(
    value=110,
    min=50,
    max=150,
    step=1,
    description='Ausübungspreis:',
    disabled=False
),
                               r = widgets.BoundedFloatText(
    value=0.01,
    min=0,
    max=1,
    step=0.01,
    description='Zinssatz:',
    disabled=False
),
                               sigma = widgets.BoundedFloatText(
    value=0.05,
    min=0.01,
    max=1,
    step=0.01,
    description='Volatilität:',
    disabled=False
))
                               


# Visualization

# Optionspreise abhängig vom Ausübungspreis:

In [31]:
interactive_plot1

interactive(children=(BoundedFloatText(value=100.0, description='Basispreis:', max=150.0, min=50.0, step=1.0),…

# Preise zweier Calls abhängig vom Basis- und Ausübungspreis

In [32]:
interactive_plot2

interactive(children=(BoundedFloatText(value=1.0, description='Laufzeit1:', max=2.0, min=0.1, step=0.1), Bound…

# Preis zweier Calls abhängig von Volatilität und Laufzeit

In [33]:
interactive_plot3

interactive(children=(BoundedFloatText(value=100.0, description='Basispreis1:', max=150.0, min=50.0, step=1.0)…

# Delta eines Calls abhängig von Basispreis und Laufzeit

In [34]:
interactive_plot4

interactive(children=(BoundedFloatText(value=110.0, description='Ausübungspreis:', max=150.0, min=50.0, step=1…

# Gamma eines Calls abhängig von Basispreis und Laufzeit

In [35]:
interactive_plot5

interactive(children=(BoundedFloatText(value=110.0, description='Ausübungspreis:', max=150.0, min=50.0, step=1…

# Vega eines Calls abhängig von Basispreis und Laufzeit

In [36]:
interactive_plot6

interactive(children=(BoundedFloatText(value=110.0, description='Ausübungspreis:', max=150.0, min=50.0, step=1…

# Theta eines Calls abhängig von Basispreis und Laufzeit

In [37]:
interactive_plot7

interactive(children=(BoundedFloatText(value=110.0, description='Ausübungspreis:', max=150.0, min=50.0, step=1…

# Rho eines Calls abhängig von Basispreis und Laufzeit

In [38]:
interactive_plot8

interactive(children=(BoundedFloatText(value=110.0, description='Ausübungspreis:', max=150.0, min=50.0, step=1…

# Tests

In [39]:
print(phi(x = 0,mu = 0 , sigma = 1)*math.sqrt(2*math.pi))

1.0


In [40]:
S = 100
K = 110
t = 1
r = 0.02
sigma = 0.10
print("Optionspreis:" + str(calcCall(S,K, t,r, sigma)))
print("Delta: " + str(calcDeltaCall(S,K, t,r, sigma)))
print("Gamma: " + str(calcGammaCall(S,K, t,r, sigma)))
print("Vega: " + str(calcVegaCall(S,K, t,r, sigma)))
print("Rho " + str(calcRhoCall(S,K, t,r, sigma)))
print("Theta: " + str(calcThetaCall(S,K, t,r, sigma)))

Optionspreis:1.35
Delta: 0.24099615585831619
Gamma: 0.031157518596646044
Vega: 31.157518596646046
Rho 22.745877220313464
Theta: -2.012793474238572


In [41]:
#test CalcDs
print(calcDs(S = 100, K = 100, r = 1, sigma = math.sqrt(2), t=1))
print(calcDs(S = 1000,K = 1, t = 1, r = 0.01, sigma = 0.05))

(1.414213562373095, -2.220446049250313e-16)
(138.38010557964273, 138.33010557964272)


In [42]:
#test calcCall
print(calcCall(S = 1000,K = 1, t = 1, r = 0.01, sigma = 0.05))
print(calcCall(S = 100,K = 110, t = 1, r = 0.01, sigma = 0.00001))

999.01
0.0


In [43]:
print(calcCall(S = 100,K = 100, t = 1, r = 0.01, sigma = 0.05))

2.52
